# Getting the Data

In [3]:
from typing import *
from pandas_profiling import ProfileReport
import pandas as pd
from mip import *

Using Python-MIP package version 1.7.3


In [25]:
def get_data():
    df = pd.read_csv('all_wines_cleaned.csv').sort_values(['Avaliações'], ascending=False)
    #Incluir somente os vinhos disponíveis e com alguma pontuação, pois essas serão nossas variáveis de decisão principais
    data = df[df.Preço_Normal.notna() & df.Pontos_Total != 0].copy().reset_index(drop=True)
    data = data.drop(['Tipo_Cat', 'Preços_Cat', 'Pontuação_Cat', 'Estoque_Cat', 'Pontos_Total', 'Preço_Sócio'], axis=1)
    data.set_index('Nome', inplace=True)
    return data

In [26]:
data = get_data()

Os vinhos do catálogo possuem cerca de 15 por cento de desconto para os associados, i.e. `Preço_Normal ~ 0.85 * Preço_Sócio`

Considerando como uma aproximação que no Preço Normal incida aproximadamente 25% do valor do Preço de Custo do Vinho, i.e. 10% do Preço_Sócio.

Como o Preço Sócio não adiciona informação para o nosso problema eliminamos essa variável

In [27]:
data['Custo'] = data['Preço_Normal'] / 1.25

Em problemas de otimização devemos ter somente variáveis numéricas,i.e. reais, ordinais e booleanas. Algumas variáveis numéricas no entanto tem muitos valores nulos, como Decantação por exemplo, outras como Safra na verdade são variáveis categóricas no formato de número. Não iremos utilizá-las em nosso problema de otimização.

In [94]:
NUM: List[str] = ['Custo', 'Preço_Normal', 'Pontuação', 'Avaliações', 'Temperatura', 'Teor_Alcoólico', 'Potencial_Guarda']

Existem outras variáveis categóricas com baixa cardinalidade de potencialmente possam ser transformadas e utilizadas.

In [95]:
CAT: List[str] = ['Tipo', 'País', 'Puro']

In [148]:
df = data[NUM + CAT].copy()

In [149]:
df.isnull().sum()

Custo                 0
Preço_Normal          0
Pontuação             0
Avaliações            0
Temperatura           8
Teor_Alcoólico        5
Potencial_Guarda      0
Tipo                  2
País                122
Puro                  0
dtype: int64

In [150]:
df['Temperatura'] = df.Temperatura.fillna(df.Temperatura.mean())
df['Teor_Alcoólico'] = df.Temperatura.fillna(df.Temperatura.mean())
df.isnull().any()

Custo               False
Preço_Normal        False
Pontuação           False
Avaliações          False
Temperatura         False
Teor_Alcoólico      False
Potencial_Guarda    False
Tipo                 True
País                 True
Puro                False
dtype: bool

# Definição do Problema
A variável de decisão é composta pelos vinhos. O Orcamento será utilizado como _constraint_, no entanto é ajustável.

In [33]:
BUDGET: int = 1000000
VINHOS = df.index.to_list()
N = range(len(VINHOS))

## Helper Functions

In [205]:
def optimize_model(model:Model, max_gap: float=0.05, max_seconds: int=300)->Tuple[str, Dict[str, float]]:
    model.max_gap = max_gap
    status = model.optimize(max_seconds=max_seconds)
    if status == OptimizationStatus.OPTIMAL:
        solution = 'OPTIMAL'
        print('Optimal solution cost {} found'.format(model.objective_value))
    elif status == OptimizationStatus.FEASIBLE:
        solution = 'FEASIBLE'
        print('sol.cost {} found, best possible: {}'.format(model.objective_value, model.objective_bound))
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        solution = 'UNFEASIBLE'
        print('No feasible solution found, lower bound is: {}'.format(model.objective_bound))
    if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
        return {v.name:v.x for v in model.vars if abs(v.x) > 1e-6}, solution
    return None, solution

def safra_stats(df: pd.DataFrame)-> None:
    print(f'Nº de Vinhos Distintos: {df.shape[0]}')
    print(f'Mediana do Preço: {df.Preço_Normal.median()}')
    print(f'Mediana da Pontuação: {df.Pontuação.median()}')
    print(f'Mediana do nº de Avaliações: {df.Avaliações. median()}')
    print(f'Total do Orçamento Utilizado: {df.Custo.sum() * 1000:.2f}')
    print(f'Distribuição da Pontuação: \n{df.Pontuação.value_counts(ascending=False)}')    
    print(f'Distribuição do Potencial de Guarda: \n{df.Potencial_Guarda.value_counts(ascending=False)}')    

In [225]:
def run_model(variable: List, 
                 constraints: Dict[str, Tuple[float, float]],
                 sense: str = 'MAX', 
                 is_uniform: bool=True, 
                 ub: int=1000):
    """Constructs a MIP model to optimize variable subject to `constraints. Sense of Optimization defaults to MAX
    If `is_uniform` is True, it turns the main variable `vinhos` into a binary decision variable
    which if True is equivalent to one batch of `ub` 
    """
    m = Model(sense=sense)
    
    if is_uniform:
        wines = [m.add_var(name=vinho, var_type=BINARY) for vinho in VINHOS]
    else:
        wines = [m.add_var(name=vinho, var_type=INTEGER, lb=0, ub=ub) for vinho in VINHOS]
        
    ALL = set(NUM).union(CAT)
    
    assert variable in ALL, f'A variável de decisão deve pertencer ao conjunto {ALL}' 
    assert set(constraints.keys()).issubset(ALL), f'As variáveis do problema devem pertencer do conjunto {ALL}'
    
    var = df[variable].to_list()
    
    #Objective Function
    m += xsum(wines[i] * var[i] for i in N)
    
    const = constraints.copy()
    
    budget = BUDGET / ub if is_uniform else BUDGET
    
    # Main constraint is budget
    custo = df['Custo'].to_list()
    m += xsum(wines[i] * custo[i] for i in N) <= budget
    m += xsum(wines[i] * custo[i] for i in N) >= budget - df.Custo.min() + 1e-6

    cat = set(constraints.keys()).intersection(CAT)
    num = set(constraints.keys()).intersection(NUM)

    cat = {k: constraints[k] for k in cat}
    num = {k: constraints[k] for k in num}
    
    if not all([isinstance(val, tuple) and len(val)==2 for val in num.values()]):
        raise ValueError(f'Os constraints {num.values()} devem ser tuplas com os valores (mínimo, máximo) das variáveis')    
    
    if len(cat):
        for col, col_dict in cat.items():
            if not all([isinstance(val, tuple) and len(val) == 2 for val in col_dict.values()]):
                raise ValueError(f'Os constraints {col_dict.values()} devem ser tuplas com os valores (mínimo, máximo) das variáveis')            
            
            df_dummies = pd.get_dummies(df[col])
            for k, (minimo, maximo) in col_dict.items():
                assert minimo >= 0 and maximo <= 1, f'As variáveis categóricas são proporcionais e devem estar no intervalo [0,1]'
                m += xsum(wines[i] * df_dummies[k].to_list()[i] for i in N) >= minimo * xsum(wines[i] for i in N)
                m += xsum(wines[i] * df_dummies[k].to_list()[i] for i in N) <= maximo * xsum(wines[i] for i in N)
    for i in N:        
        for key, (minimo, maximo) in num.items():
            m += wines[i] * df[key].to_list()[i] >= minimo * wines[i]
            m += wines[i] * df[key].to_list()[i] <= maximo * wines[i]
            
    solution, status = optimize_model(m)
    
    if solution is not None:
        multiplier = ub if is_uniform else 1
        result = data.loc[solution.keys()].copy()
        result['Quantidade'] = [val * multiplier for val in solution.values()]
        return result, status
    
    print('!!!No result was found for the Optimization Problem with the Variable and constraints provided!!!')
    return None, status

Maximizar o Preço sem Constraint

In [213]:
var = 'Preço_Normal'
const = {'Pontuação': (0, 5)}
resultado, status = run_model(var, const, is_uniform=False, ub=12000)
resultado[["Quantidade", 'Custo', 'Preço_Normal', 'Pontuação', 'Avaliações']]

Optimal solution cost 1249979.04 found


,Quantidade,Custo,Preço_Normal,Pontuação,Avaliações
Nome,,,,,
Calyptra Assemblage Gran Reserva 2014,6729,148.608,185.76,4.0,1.0


Minimizar o Preço sem Constraint

In [214]:
var = 'Preço_Normal'
const = {'Pontuação': (0, 5)}
resultado, status = run_model(var, const, is_uniform=False, ub=12000, sense='MIN')
resultado[["Quantidade", 'Custo', 'Preço_Normal', 'Pontuação', 'Avaliações']]

Optimal solution cost 1249974.8399999999 found


,Quantidade,Custo,Preço_Normal,Pontuação,Avaliações
Nome,,,,,
Viña Cosos D.O. Campo de Borja Garnacha Syrah 2018,12000,39.920,49.90,4.0,254.0
Terralis Cabernet Merlot 2018,11157,22.496,28.12,4.0,76.0
Espumante Baby Chandon Réserve Brut 187 ml.,12000,22.496,28.12,4.0,14.0


Minimizar Preço com Constraint de Pontuação e Avaliações e a distribuição dos vinhos uniforme com 1000 garrafas

In [215]:
var = 'Preço_Normal'
const = {'Pontuação': (4, 5), 'Avaliações': (10, 10000)}
resultado, status = run_model(var, const, sense='MIN')
resultado[["Quantidade", 'Custo', 'Pontuação', 'Avaliações']]

Optimal solution cost 1224.24 found


,Quantidade,Custo,Pontuação,Avaliações
Nome,,,,
Praça dos Marqueses Escolha I.G. Beira Atlântico 2018,1000,27.200,4.0,339.0
Toro Loco D.O.P. Utiel-Requena Tinto Superior 2018,1000,23.432,4.5,261.0
Champagne Montaudon Brut,1000,173.080,4.5,156.0
Sueño D.O. Ribera Del Júcar Tempranillo 2013,1000,75.120,4.0,115.0
Espumante Chandon Réserve Brut,1000,65.784,4.5,79.0
Espumante Real De Aragón D.O. Cava Rosé Brut,1000,57.320,4.0,58.0
Portas da Herdade Tinto 2018,1000,27.200,4.0,53.0
Tenuta Sant`Antonio Scaia Bianco 2017,1000,62.968,4.5,51.0
Pavillon Saint Pierre Réserve 2016,1000,47.904,4.0,47.0


* Minimizar Preço 
 * Pontuação entre (4,5)
 * No Mínimo 10 Avaliações
 * Distribuição Uniforme de 1000 garrafas por vinho
 * Metade dos Vinhos Puros (Somente 1 uva)

In [216]:
var = 'Preço_Normal'
const = {'Pontuação': (4,5), 'Avaliações': (10, 10000), 'Puro':{1: (0.5, 0.5)}}
resultado, status = run_model(var, const, sense='MIN', ub=2000)
print(f'Orçamento Utilizado: {sum(r.Quantidade * r.Custo for r in resultado.itertuples()):.2f}')
resultado[["Quantidade", 'Custo', 'Pontuação', 'Avaliações', 'Puro']]

Optimal solution cost 599.29 found
Orçamento Utilizado: 958864.00


,Quantidade,Custo,Pontuação,Avaliações,Puro
Nome,,,,,
Espumante Veuve D`Argent Blanc De Blancs Brut,2000,34.728,4.0,360.0,0
Urmeneta Carménère 2018,2000,25.320,4.0,59.0,1
Finca Traversa Tannat 2018,2000,24.376,4.0,37.0,1
Natana Cuvée Rouge 2017,2000,34.728,4.0,29.0,0
Beaupont,2000,24.376,4.0,27.0,1
Urmeneta Chardonnay 2019,2000,25.320,4.0,24.0,1
Partridge Selección de Barricas Blend 2015,2000,91.200,4.5,23.0,0
Espumante Chandon Passion Rosé Demi-Sec,2000,94.400,5.0,16.0,0
Tenuta Sant`Antonio Scaia Bianco 2018,2000,66.728,4.5,13.0,0


In [217]:
var = 'Preço_Normal'
const = {'Pontuação': (4,5), 'Avaliações': (10, 10000), 'Puro':{1: (0.33, 0.66)}}
resultado, status = run_model(var, const, sense='MIN', ub=2000)
print(f'Orçamento Utilizado: {sum(r.Quantidade * r.Custo for r in resultado.itertuples()):.2f}')
resultado[["Quantidade", 'Custo', 'Pontuação', 'Avaliações', 'Puro']]

Optimal solution cost 599.2499999999999 found
Orçamento Utilizado: 958800.00


,Quantidade,Custo,Pontuação,Avaliações,Puro
Nome,,,,,
Esteban Martín Reserva D.O.P. Cariñena 2014,2000,46.024,4.5,67.0,0
La Mora D.O.C. Maremma Toscana Rosso 2015,2000,46.968,4.5,60.0,0
Terralis Syrah Malbec 2018,2000,20.608,4.0,55.0,0
Campo Al Moro 2016,2000,29.080,4.0,37.0,0
Marianne Selena Blend 2015,2000,46.968,4.0,29.0,0
Beaupont,2000,24.376,4.0,27.0,1
La Chamiza Malbec 2018,2000,30.024,4.0,25.0,1
Espumante Don Juan,2000,50.320,4.5,24.0,1
Espumante Chandon Rosé Brut,2000,80.848,4.5,14.0,0


In [218]:
var = 'Preço_Normal'
const = {'Pontuação': (4,5), 'Avaliações': (10, 10000), 'Puro':{1: (0.5, 1)}, 'País':{'França': (0.33, 1)}}
resultado, status = run_model(var, const, sense='MIN')
print(f'Orçamento Utilizado: {sum(r.Quantidade * r.Custo for r in resultado.itertuples()):.2f}')
resultado[["Quantidade", 'Custo', 'Pontuação', 'Avaliações', 'Puro', 'País']]

Optimal solution cost 1224.27 found
Orçamento Utilizado: 979416.00


,Quantidade,Custo,Pontuação,Avaliações,Puro,País
Nome,,,,,,
Viña Cosos D.O. Campo de Borja Garnacha Syrah 2018,1000,39.920,4.0,254.0,0,Espanha
Espumante Veuve D'Argent Premium Edition Ice Blanc,1000,41.320,4.5,39.0,1,França
Finca Traversa Tannat 2018,1000,24.376,4.0,37.0,1,Uruguai
Urmeneta Cabernet Sauvignon 2018,1000,28.144,4.5,35.0,1,NaN
Almaviva EPU 2015,1000,399.200,5.0,31.0,0,NaN
L'Ostal Cazes Eclipse Syrah 2017,1000,53.552,4.0,29.0,1,França
Corello I.G.P. Puglia Negroamaro 2018,1000,28.144,4.5,28.0,1,Itália
Beaupont,1000,24.376,4.0,27.0,1,França
Enclos du Wine Hunter A.O.C. Bordeaux Rosé 2018,1000,62.960,4.0,22.0,0,França


In [226]:
var = 'Preço_Normal'
const = {'Pontuação': (4,5), 
         'Avaliações': (10, 10000), 
         'Puro':{1: (0.33, 1)}, 
         'País': {'França': (0.33, 1), 'Argentina': (0.2, 1)},
         'Tipo': {'Frisante': (0.2, 1), 'Tinto': (0.5, 1)}}
         #'Potencial_Guarda': (3, 50)}
resultado, status = run_model(var, const, sense='MIN', is_uniform=False)
print(f'Orçamento Utilizado: {sum(r.Quantidade * r.Custo for r in resultado.itertuples()):.2f}')
resultado[["Quantidade", 'Custo', 'Pontuação', 'Avaliações', 'Puro', 'País', 'Tipo', 'Potencial_Guarda']]

Optimal solution cost 1249993.0 found
Orçamento Utilizado: 999994.40


,Quantidade,Custo,Pontuação,Avaliações,Puro,País,Tipo,Potencial_Guarda
Nome,,,,,,,,
Chant du Coq Sélection Jean Vincent Bideau 2016,4,78.616,4.5,72.0,1,França,Tinto,5.0
Frisante Porta Soprana D.O.C. Lambrusco Grasparossa di Castelvetro Amabile Suave,1000,38.496,4.5,48.0,1,Itália,Frisante,2.0
Domaine de L'Ostal Cazes Estibals A.O.C. Minervois 2014,352,99.600,4.5,45.0,0,França,Tinto,7.0
Frisante Porta Soprana D.O.C. Lambrusco Salamino di Santa Croce Tinto Amabile Suave,1000,31.904,4.0,42.0,1,Itália,Frisante,2.0
Frisante Porta Soprana D.O.C. Lambrusco di Sorbara Tinto Meio Seco,350,39.432,4.0,41.0,1,Itália,Frisante,1.0
V9 Gran Reserva Single Vineyard Cabernet Sauvignon 2017,1000,48.848,4.5,35.0,0,NaN,Tinto,7.0
Partridge Selección de Barricas Blend 2015,1000,91.200,4.5,23.0,0,Argentina,Tinto,8.0
Fortant de France Terroir d'Altitude Syrah 2017,1000,51.672,4.0,19.0,1,França,Tinto,6.0
Louis Bouillot A.O.C. Crémant de Bourgogne Rosé Brut,522,95.920,4.0,18.0,0,França,Espumante,3.0


In [222]:
%debug

> <ipython-input-212-f7b7f530029b>(20)run_model()
     18 
     19     assert variable in ALL, f'A variável de decisão deve pertencer ao conjunto {ALL}'
---> 20     assert set(constraints.keys()).issubset(ALL), f'As variáveis do problema devem pertencer do conjunto {ALL}'
     21 
     22     var = df[variable].to_list()



ipdb>  constraints


({'Pontuação': (4, 5), 'Avaliações': (10, 10000), 'Puro': {1: (0.5, 1)}, 'País': {'França': (0.33, 1), 'Argentina': (0.2, 1)}, 'Tipo': {'Frisante': (0.2, 1), 'Tinto': (0.5, 1)}},)


ipdb>  exit
